# Voi original transformation 

The scope of this script is to assign a trip ID to VOI's original data file and ensure it matches with the pre-query transformation file

In [35]:
import pandas as pd
import requests
import numpy as np
from datetime import datetime, timedelta
import haversine as hs
from haversine import Unit

In [36]:
df_original = pd.read_csv('folder/file.csv')

In [37]:
df = df_original

## 1. Global cleaning 

In [38]:
#1 dropping End-Time & Start Date
df=df.drop(['START_DATE'], axis=1)

#2 renaming columns to match structure of requests.csv
df=df.rename(columns={"START_TIME":"treq","START_LONGITUDE":"origin_x","START_LATITUDE":"origin_y","END_LONGITUDE":"destination_x","END_LATITUDE":"destination_y",})

In [39]:
#3 drop out of bound trips
minLong = 11.8735
minLat = 57.6265
maxLong = 12.0427
maxLat = 57.7712

len1=len(df)
len1
df.drop(df.loc[df['origin_x']<minLong].index, inplace=True)
df.drop(df.loc[df['origin_y']<minLat].index, inplace=True)
df.drop(df.loc[df['destination_x']<minLong].index, inplace=True)
df.drop(df.loc[df['destination_y']<minLat].index, inplace=True)
df.drop(df.loc[df['origin_x']>maxLong].index, inplace=True)
df.drop(df.loc[df['origin_y']>maxLat].index, inplace=True)
df.drop(df.loc[df['destination_x']>maxLong].index, inplace=True)
df.drop(df.loc[df['destination_y']>maxLat].index, inplace=True)
len2=len(df)

print("Rows removed:", len1-len2)


Rows removed: 258


In [40]:
#5. Time and duration: formatting start and end to datetime, incrementing to UTC+1 since we assume that GTFS data is saved in Gothenburg's local time (check with Tobias), AND calculating duration
df['treq'] = pd.to_datetime(df['treq'], format='%Y-%m-%d %H:%M:%S')
df['treq'] = pd.to_datetime(df['treq'].dt.strftime('%Y-%m-%d %H:%M:%S'))

df['END_TIME'] = pd.to_datetime(df['END_TIME'], format='%Y-%m-%d %H:%M:%S')
df['END_TIME'] = pd.to_datetime(df['END_TIME'].dt.strftime('%Y-%m-%d %H:%M:%S'))


df['duration_in_m']=df['END_TIME']-df['treq']
df['duration_in_m']=(df.duration_in_m.dt.total_seconds() / 60)
df

,TRIP_ID,origin_x,origin_y,destination_x,destination_y,treq,END_TIME,duration_in_m
0,d6abf88f-5a45-5845-9b76-fba4970c2a68,11.944810,57.698154,11.940518,57.692970,2022-04-27 19:56:15,2022-04-27 20:03:41,7.433333
1,3c186cb9-15ef-54b6-96a8-0e8eea1299a0,11.984405,57.706665,11.996304,57.691113,2022-04-27 10:25:04,2022-04-27 10:33:09,8.083333
2,f11bf258-bddd-50a2-bee1-37b36590728b,11.991008,57.705502,11.980876,57.703041,2022-04-27 16:21:11,2022-04-27 16:24:52,3.683333
3,8cec0a04-aef3-5ed0-af65-f1e0c4a7d5b0,11.944882,57.717285,11.967620,57.707108,2022-04-27 06:45:57,2022-04-27 06:55:21,9.400000
4,1133b471-3459-5de4-b87c-cadb2aec4352,11.963046,57.702522,11.963737,57.700203,2022-04-27 10:22:09,2022-04-27 10:24:28,2.316667
...,...,...,...,...,...,...,...,...
1040626,788203bf-f44a-55fd-a9a7-56d29ba19564,11.928447,57.709794,11.934595,57.705265,2022-10-27 11:32:25,2022-10-27 11:36:07,3.700000
1040627,9ddd1063-37f2-5d02-b6e5-60e6464230b0,11.956453,57.699149,11.994070,57.699961,2022-10-27 20:10:10,2022-10-27 20:23:45,13.583333
1040628,1e291ded-7f32-5164-a651-3f6a65d3e567,11.934535,57.705272,11.928570,57.709841,2022-10-27 11:22:28,2022-10-27 11:26:13,3.750000
1040629,37cd82fe-47c6-5c09-acf6-3dfc3f2a6ff0,12.000091,57.681917,11.981809,57.688730,2022-10-27 10:16:17,2022-10-27 10:22:44,6.450000


In [172]:
## code to remove rows that have distance and duration under a certain value?

In [41]:

##=============================ASSIGNING ID AFTER ALL CLEANING IS MADE
## Sort oldest trip first, and assign id. ID is being assigned BEFORE the dataset is split by schedule to ensure cardinality if the datasets need to be concatenated 
df.sort_values(by='treq', ascending=True, inplace=True)
df['TRIP_ID']=np.arange(len(df))

In [45]:
#6 split summer and winter trips. IMPORTANT. VOI's ORIGINAL DATA MUST ALSO BE SPLIT TO RETAIN ROW-WISE INDEX IDENTIFIERS
start_date = '2022-06-19' #day summer schedule starts
end_date   = '2022-08-20' #day summer schedule ends

df_summer = df.query('treq > @start_date and treq < @end_date')
df_summer.to_csv('VoiData_CleanedIndexed_Summer.csv', index=False)

In [46]:
#6 dataframe for winter trips
start_date = '2022-06-19' #day summer schedule starts
end_date   = '2022-08-20' #day summer schedule ends

df_winter1 = df.query('treq < @start_date') #all dates before summer
df_winter2 = df.query('treq > @end_date') #all dates after summer
df_winter = pd.concat([df_winter1,df_winter2])
df_winter.to_csv('VoiData_CleanedIndexed_Winter.csv', index=False)

In [49]:
df_winter

,TRIP_ID,origin_x,origin_y,destination_x,destination_y,treq,END_TIME,duration_in_m
0,0,11.946541,57.723145,11.941060,57.721714,2022-01-08 00:00:40,2022-01-01 00:03:47,3.116667
1,1,11.978535,57.697792,11.970809,57.691029,2022-01-08 00:01:01,2022-01-01 00:08:23,7.366667
2,2,11.990700,57.712872,12.028315,57.711536,2022-01-08 00:01:41,2022-01-01 00:25:35,23.900000
3,3,11.983342,57.688713,11.985471,57.687702,2022-01-08 00:01:50,2022-01-01 00:02:54,1.066667
4,4,11.952630,57.691753,11.931505,57.690220,2022-01-08 00:02:57,2022-01-01 00:12:00,9.050000
...,...,...,...,...,...,...,...,...
1040368,1040368,11.982319,57.698499,11.962712,57.695212,2022-01-07 00:13:55,2022-11-11 00:21:53,7.966667
1040369,1040369,11.991790,57.705063,11.978468,57.693824,2022-01-07 00:14:11,2022-11-11 00:23:13,9.033333
1040370,1040370,11.971477,57.709383,11.940520,57.726088,2022-01-07 00:14:46,2022-11-11 00:29:07,14.350000
1040371,1040371,11.971631,57.700933,11.952006,57.694424,2022-01-07 00:24:12,2022-11-11 00:33:26,9.233333


## 2. Transformation for Querying

In [50]:
#7 assigning appropriate day of the week 

#probably a runner-up as the world's most inneficient piece of code 

monday = '2022-01-03 '
tuesday = '2022-01-04 '
wednesday = '2022-01-05 '
thursday = '2022-01-06 '
friday = '2022-01-07 '
saturday = '2022-01-08 '
sunday = '2022-01-09 '


df_winter['treq'] = pd.to_datetime(df_winter['treq'], format='%Y-%m-%d %H:%M:%S')
df_winter['treq'] = pd.to_datetime(df_winter['treq'].dt.strftime('%Y-%m-%d %H:%M:%S'))

df_winter['treqDayName']=df_winter['treq'].dt.day_name()
df_winter['time'] = [d.time() for d in df_winter['treq']]

df_winterMon = df_winter.loc[df_winter['treqDayName'] == 'Monday']
df_winterMon['treq'] = pd.to_datetime(df_winterMon['treq'])
df_winterMon['mondayDate'] = monday
df_winterMon['treq'] = df_winterMon['mondayDate'].astype(str) + df_winterMon['time'].astype(str)
df_winterMon=df_winterMon.drop(['treqDayName','time','mondayDate'], axis=1)

df_winterTue = df_winter.loc[df_winter['treqDayName'] == 'Tuesday']
df_winterTue['treq'] = pd.to_datetime(df_winterTue['treq'])
df_winterTue['tuesdayDate'] = tuesday
df_winterTue['treq'] = df_winterTue['tuesdayDate'].astype(str) + df_winterTue['time'].astype(str)
df_winterTue=df_winterTue.drop(['treqDayName','time','tuesdayDate'], axis=1)

df_winterWed= df_winter.loc[df_winter['treqDayName'] == 'Wednesday']
df_winterWed['treq'] = pd.to_datetime(df_winterWed['treq'])
df_winterWed['wednesdayDate'] = wednesday
df_winterWed['treq'] = df_winterWed['wednesdayDate'].astype(str) + df_winterWed['time'].astype(str)
df_winterWed=df_winterWed.drop(['treqDayName','time','wednesdayDate'], axis=1)

df_winterThur= df_winter.loc[df_winter['treqDayName'] == 'Thursday']
df_winterThur['treq'] = pd.to_datetime(df_winterThur['treq'])
df_winterThur['thursdayDate'] = thursday
df_winterThur['treq'] = df_winterThur['thursdayDate'].astype(str) + df_winterThur['time'].astype(str)
df_winterThur=df_winterThur.drop(['treqDayName','time','thursdayDate'], axis=1)

df_winterFri= df_winter.loc[df_winter['treqDayName'] == 'Friday']
df_winterFri['treq'] = pd.to_datetime(df_winterFri['treq'])
df_winterFri['fridayDate'] = friday
df_winterFri['treq'] = df_winterFri['fridayDate'].astype(str) + df_winterFri['time'].astype(str)
df_winterFri=df_winterFri.drop(['treqDayName','time','fridayDate'], axis=1)

df_winterSat= df_winter.loc[df_winter['treqDayName'] == 'Saturday']
df_winterSat['treq'] = pd.to_datetime(df_winterSat['treq'])
df_winterSat['saturdayDate'] = saturday
df_winterSat['treq'] = df_winterSat['saturdayDate'].astype(str) + df_winterSat['time'].astype(str)
df_winterSat=df_winterSat.drop(['treqDayName','time','saturdayDate'], axis=1)

df_winterSun= df_winter.loc[df_winter['treqDayName'] == 'Sunday']
df_winterSun['treq'] = pd.to_datetime(df_winterSun['treq'])
df_winterSun['sundayDate'] = sunday
df_winterSun['treq'] = df_winterSun['sundayDate'].astype(str) + df_winterSun['time'].astype(str)
df_winterSun=df_winterSun.drop(['treqDayName','time','sundayDate'], axis=1)


df_winter = pd.concat([df_winterMon,df_winterTue,df_winterWed,df_winterThur,df_winterFri,df_winterSat,df_winterSun])
df_winter.sort_values(by='TRIP_ID', ascending=True, inplace=True)

C:\Users\ITM Nord Lånedator\AppData\Local\Temp\ipykernel_2292\2117655055.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_winterMon['treq'] = pd.to_datetime(df_winterMon['treq'])
C:\Users\ITM Nord Lånedator\AppData\Local\Temp\ipykernel_2292\2117655055.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_winterMon['mondayDate'] = monday
C:\Users\ITM Nord Lånedator\AppData\Local\Temp\ipykernel_2292\2117655055.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

In [52]:
#7 assigning appropriate day of the week 

#probably a runner-up as the world's most inneficient piece of code 
monday = '2022-06-27 '
tuesday = '2022-06-28 '
wednesday = '2022-06-29 '
thursday = '2022-06-30 '
friday = '2022-07-01 '
saturday = '2022-07-02 '
sunday = '2022-07-03 '


df_summer['treq'] = pd.to_datetime(df_summer['treq'], format='%Y-%m-%d %H:%M:%S')
df_summer['treq'] = pd.to_datetime(df_summer['treq'].dt.strftime('%Y-%m-%d %H:%M:%S'))

df_summer['treqDayName']=df_summer['treq'].dt.day_name()
df_summer['time'] = [d.time() for d in df_summer['treq']]

df_summerMon = df_summer.loc[df_summer['treqDayName'] == 'Monday']
df_summerMon['treq'] = pd.to_datetime(df_summerMon['treq'])
df_summerMon['mondayDate'] = monday
df_summerMon['treq'] = df_summerMon['mondayDate'].astype(str) + df_summerMon['time'].astype(str)
df_summerMon=df_summerMon.drop(['treqDayName','time','mondayDate'], axis=1)

df_summerTue = df_summer.loc[df_summer['treqDayName'] == 'Tuesday']
df_summerTue['treq'] = pd.to_datetime(df_summerTue['treq'])
df_summerTue['tuesdayDate'] = tuesday
df_summerTue['treq'] = df_summerTue['tuesdayDate'].astype(str) + df_summerTue['time'].astype(str)
df_summerTue=df_summerTue.drop(['treqDayName','time','tuesdayDate'], axis=1)

df_summerWed= df_summer.loc[df_summer['treqDayName'] == 'Wednesday']
df_summerWed['treq'] = pd.to_datetime(df_summerWed['treq'])
df_summerWed['wednesdayDate'] = wednesday
df_summerWed['treq'] = df_summerWed['wednesdayDate'].astype(str) + df_summerWed['time'].astype(str)
df_summerWed=df_summerWed.drop(['treqDayName','time','wednesdayDate'], axis=1)

df_summerThur= df_summer.loc[df_summer['treqDayName'] == 'Thursday']
df_summerThur['treq'] = pd.to_datetime(df_summerThur['treq'])
df_summerThur['thursdayDate'] = thursday
df_summerThur['treq'] = df_summerThur['thursdayDate'].astype(str) + df_summerThur['time'].astype(str)
df_summerThur=df_summerThur.drop(['treqDayName','time','thursdayDate'], axis=1)

df_summerFri= df_summer.loc[df_summer['treqDayName'] == 'Friday']
df_summerFri['treq'] = pd.to_datetime(df_summerFri['treq'])
df_summerFri['fridayDate'] = friday
df_summerFri['treq'] = df_summerFri['fridayDate'].astype(str) + df_summerFri['time'].astype(str)
df_summerFri=df_summerFri.drop(['treqDayName','time','fridayDate'], axis=1)

df_summerSat= df_summer.loc[df_summer['treqDayName'] == 'Saturday']
df_summerSat['treq'] = pd.to_datetime(df_summerSat['treq'])
df_summerSat['saturdayDate'] = saturday
df_summerSat['treq'] = df_summerSat['saturdayDate'].astype(str) + df_summerSat['time'].astype(str)
df_summerSat=df_summerSat.drop(['treqDayName','time','saturdayDate'], axis=1)

df_summerSun= df_summer.loc[df_summer['treqDayName'] == 'Sunday']
df_summerSun['treq'] = pd.to_datetime(df_summerSun['treq'])
df_summerSun['sundayDate'] = sunday
df_summerSun['treq'] = df_summerSun['sundayDate'].astype(str) + df_summerSun['time'].astype(str)
df_summerSun=df_summerSun.drop(['treqDayName','time','sundayDate'], axis=1)


df_summer = pd.concat([df_summerMon,df_summerTue,df_summerWed,df_summerThur,df_summerFri,df_summerSat,df_summerSun])
df_summer.sort_values(by='TRIP_ID', ascending=True, inplace=True)
df_summer

C:\Users\ITM Nord Lånedator\AppData\Local\Temp\ipykernel_2292\794768673.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_summerMon['treq'] = pd.to_datetime(df_summerMon['treq'])
C:\Users\ITM Nord Lånedator\AppData\Local\Temp\ipykernel_2292\794768673.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_summerMon['mondayDate'] = monday
C:\Users\ITM Nord Lånedator\AppData\Local\Temp\ipykernel_2292\794768673.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

,TRIP_ID,origin_x,origin_y,destination_x,destination_y,treq,END_TIME,duration_in_m
529866,529866,11.975588,57.708882,12.000563,57.714166,2022-07-03 03:00:59,2022-06-19 03:07:53,6.900000
529867,529867,11.986027,57.706913,11.985992,57.703012,2022-07-03 03:02:18,2022-06-19 03:11:01,8.716667
529868,529868,11.992002,57.691469,11.993202,57.689584,2022-07-03 03:03:21,2022-06-19 03:05:55,2.566667
529869,529869,11.995759,57.695672,12.007040,57.704726,2022-07-03 03:03:51,2022-06-19 03:09:36,5.750000
529870,529870,11.965565,57.698309,11.975567,57.699879,2022-07-03 03:04:30,2022-06-19 03:15:36,11.100000
...,...,...,...,...,...,...,...,...
772138,772138,11.947095,57.695297,11.965359,57.697781,2022-07-01 19:56:24,2022-08-19 20:03:38,7.233333
772139,772139,11.924828,57.690157,11.921186,57.693514,2022-07-01 19:57:03,2022-08-19 19:58:46,1.716667
772140,772140,11.946769,57.693184,11.972190,57.684282,2022-07-01 19:58:26,2022-08-19 20:07:56,9.500000
772141,772141,11.921893,57.695215,11.959331,57.691441,2022-07-01 19:59:20,2022-08-19 20:10:05,10.750000


In [34]:
#8 export to CSV 
df_winter.to_csv('Q/georequestsWinter.csv', index=False)
df_summer.to_csv('Q/georequestsSummer.csv', index=False)